In [ ]:
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess
import pickle

print('TensorFlow version: {}'.format(tf.__version__))

In [ ]:
from db_actions import DatabaseActions as db

data = db.get_parsed()
print(data[7][0])

In [ ]:
dataset = [[data[0][0], pickle.loads(bytes.fromhex(data[0][1])), data[0][2]]]
for i in range(1, len(data)):
        dataset.append([data[i][0], pickle.loads(bytes.fromhex(data[i][1])), data[i][2]])



In [ ]:

import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

vectors_tmp = []
for record in dataset:
    vectors_tmp.append(record[1])

# make numpy array vectors
vectors = np.array(vectors_tmp)


print(vectors.shape)


In [ ]:
# print max value in vector in vectors by absolute value
def max_abs_value(vectors):
    max_value = 0
    for vector in vectors:
        max_value = max(max_value, max(abs(x) for x in vector))
    return max_value

print(max_abs_value(vectors), end= '\n' + '~' * 80 + '\n')

x_train = vectors / max_abs_value(vectors)

x_train += 1
x_train = x_train / max_abs_value(x_train) * (1 - 1 / 10 ** 5)
print(max_abs_value(x_train))


In [ ]:
y_tmp = []
for record in dataset:
    if record[2] not in [0, 1]:
        y_tmp.append(0)
    else:
        y_tmp.append(record[2])

# make numpy array vectors
y_train_t = np.array(y_tmp)
y_train = y_train_t.astype(bool)

print(y_train[1:10])



In [ ]:
print(x_train)
print(y_train)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Conv1D, GlobalMaxPooling1D, Attention, Reshape

model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024 * 4, activation='elu', input_shape=(768,)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024 * 3, activation='elu'),
        tf.keras.layers.Dense(1024 * 3, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024 * 2, activation='elu'),
        tf.keras.layers.Dense(1024 * 2, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='elu'),
        tf.keras.layers.Dense(1024, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='elu'),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(256, activation='elu'),
        tf.keras.layers.Dense(64, activation='elu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


# # Define the model architecture
# model = Sequential()
# model.add(Embedding(input_dim=1, output_dim=300, input_length=768))  # Embedding layer for word embeddings
# model.add(Conv1D(256, 5, activation='relu', padding='same'))  # Convolutional layer with 256 filters and kernel size of 5
# model.add(GlobalMaxPooling1D())  # Global max pooling layer
# # model.add(Reshape((256, 1)))  # Reshape the output to make it three-dimensional
# # model.add(LSTM(512, return_sequences=False))  # LSTM layer with 512 units
# # model.add(Attention())  # Attention layer to focus on important parts of the sequence
# model.add(Dense(256, activation='relu'))  # Dense layer with 256 neurons
# model.add(Dropout(0.5))  # Dropout layer for regularization
# model.add(Dense(128, activation='relu'))  # Additional dense layer with 128 neurons
# model.add(Dropout(0.5))  # Additional dropout layer
# model.add(Dense(1, activation='sigmoid'))  # Output layer with sigmoid activation

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# model.summary()


# Train the model
model.fit(x_train, y_train, epochs=100, verbose=1)



In [ ]:
for i in range(600, 610):
    print(i, dataset[i][2], dataset[i][0], sep= '\t', end='\t')
    print()


model.predict(x_train[600:610])
